Note that variant tags must be capitalized:

In [36]:
open Core

In [37]:
type basic_color =
   | Black | Red | Green | Yellow | Blue | Magenta | Cyan | White ;;

type basic_color =
    Black
  | Red
  | Green
  | Yellow
  | Blue
  | Magenta
  | Cyan
  | White


In [38]:
Cyan

- : basic_color = Cyan


In [39]:
[Yellow; Blue; Cyan]

- : basic_color list = [Yellow; Blue; Cyan]


In [40]:
let basic_color_to_int = function
  | Black -> 0 | Red     -> 1 | Green -> 2 | Yellow -> 3
  | Blue  -> 4 | Magenta -> 5 | Cyan  -> 6 

File "[40]", line 1, characters 25-133:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
White


val basic_color_to_int : basic_color -> int = <fun>


In [41]:
let basic_color_to_int = function
  | Black -> 0 | Red     -> 1 | Green -> 2 | Yellow -> 3
  | Blue  -> 4 | Magenta -> 5 | Cyan  -> 6 | White  -> 7 ;;

val basic_color_to_int : basic_color -> int = <fun>


In [42]:
let color_by_number number text =
    sprintf "\027[38;5;%dm%s\027[0m" number text;;

val color_by_number : int -> string -> string = <fun>


In [43]:
let blue = color_by_number (basic_color_to_int Blue) "Blue";;

val blue : string = "\027Blue\027"


In [44]:
printf "Hello %s World!\n" blue;;

- : unit = ()


In [45]:
type weight = Regular | Bold

type weight = Regular | Bold


In [46]:
  type color =
  | Basic of basic_color * weight (* basic colors, regular and bold *)
  | RGB   of int * int * int       (* 6x6x6 color cube *)
  | Gray  of int                   (* 24 grayscale levels *)

type color =
    Basic of basic_color * weight
  | RGB of int * int * int
  | Gray of int


We'll also represent this more complicated color space as a variant, but this time, the different tags will have arguments that describe the data available in each case. Note that variants can have multiple arguments, which are separated by *s:

In [47]:
 [RGB (250,70,70); Basic (Green, Regular)];;

- : color list = [RGB (250, 70, 70); Basic (Green, Regular)]


In [48]:
type color = 
|Basic of basic_color
|Bold of basic_color
| RGB of int * int * int
|Gray of int

type color =
    Basic of basic_color
  | Bold of basic_color
  | RGB of int * int * int
  | Gray of int


In [49]:
let color_to_int  = function
|Basic (basic_color) -> basic_color_to_int basic_color
|RGB (r,g,b) -> 16 + b + g * 6 + r *36
|Gray i->232+ i

File "[49]", line 1, characters 20-138:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
Bold _


val color_to_int : color -> int = <fun>


我们已经看到，类型错误指出了需要修正哪些方面来完成重构的代码。这非常有用，不过，要让它正确、可靠地发挥作用，需要采用一种合适的方式编写代码、尽可能的增加编译器百年关注你发现`bug`的机会。为了达到这个目的，有一个有用的经验：要避免模式匹配中的`catch-all`全包情况。

In [50]:
module Log_entry = struct
type t = 
{
session_id: string;
time: Time.t;
important: bool;
message: string;
}
end
;;

module Log_entry :
  sig
    type t = {
      session_id : string;
      time : Core.Time.t;
      important : bool;
      message : string;
    }
  end


In [51]:
module Log_entry = struct
type t = 
{
session_id: string;
time: Time.t;
important: bool;
message: string;
}
end
module Heartbeat = struct
type t = 
{
session_id: string;
time: Time.t;
user:string;
credentials: string;
}
end

module Logon = struct
type t = {
session_id: string;
time: Time.t;
user:string;
credentials: string;
}
end

module Log_entry :
  sig
    type t = {
      session_id : string;
      time : Core.Time.t;
      important : bool;
      message : string;
    }
  end


module Heartbeat :
  sig
    type t = {
      session_id : string;
      time : Core.Time.t;
      user : string;
      credentials : string;
    }
  end


module Logon :
  sig
    type t = {
      session_id : string;
      time : Core.Time.t;
      user : string;
      credentials : string;
    }
  end


client_message是一个`Logon`，或是一个`Log_entry`，如果我们像编写采用通用方式处理消息的代码，而不是特定于某一个固定消息类型，就需要使用类似`client_message`的类型作为洞中个不同消息的一个“总体”类型，然后可以匹配`client_message`来确定所处理的特定消息的类型。

In [52]:
type client_message = | Logon of Logon.t
|Heartbeat of Heartbeat.t
|Log_entry of Log_entry.t

type client_message =
    Logon of Logon.t
  | Heartbeat of Heartbeat.t
  | Log_entry of Log_entry.t


In [53]:
let messages_for_user user messages = 
let (user_messages,_) = 
List.fold messages ~init:([], String.Set.empty)
~f:(fun ((message, user_sessions) as acc) message ->
match message with
|Logon m ->
if m.Logon.user = user then
(message::messages, Set.add user_sessions m.Logon.session_id)
else acc
|Heartbeat _ | Log_entry _ ->
let session_id = match message with
| Logon m -> m.Logon.session_id
|Heartbeat m -> m.Heartbeat.session_id
|Log_entry m -> m.Log_entry.session_id
in
if Set.mem user_sessions session_id then
(message::messages,user_sessions) else acc)
in 
List.rev user_messages

val messages_for_user :
  string -> client_message Core.List.t -> client_message Core.List.t = <fun>


前面的代码中有一部分看起来很笨拙，即用来确定会话ID的逻辑。这个代码有些重复，它考虑了每一种可能的消息类型（包括`Logon`，而代码执行到这里实际上不可能有这种消息），并对每一种情况抽取会话ID。像着这样诸葛消息类型进行处理看起来没有必要。因为对于所有消息类型，会话ID的工作都是一样的。

In [54]:
module Log_entry = struct 
type t= {
important: bool;
message: string;
}
end

module Log_entry : sig type t = { important : bool; message : string; } end


In [55]:
module Heartbeat = struct
type t = {status_message: string;}
end

module Heartbeat : sig type t = { status_message : string; } end


In [56]:
module Logon = struct
type t = {user:string;
credentials:string}
end

module Logon : sig type t = { user : string; credentials : string; } end


In [57]:
type details = 
|Logon of Logon.t
|Heartbeat of Heartbeat.t
|Log_entry of Log_entry.t

type details =
    Logon of Logon.t
  | Heartbeat of Heartbeat.t
  | Log_entry of Log_entry.t


In [58]:
module Common = struct
type t = {session_id: string;
time: Time.t}
end

module Common : sig type t = { session_id : string; time : Core.Time.t; } end


In [59]:
let messages_for_user user messages = 
let(user_messages,_) = 
List.fold messages ~init:([], String.Set.empty)
~f:(fun ((message, user_sessions) as acc) ((common,details) as message) ->
let session_id = common.Common.session_id in
match details with
| Logon m -> if m.Logon.user = user then
(message::messages, Set.add user_sessions session_id)
else acc
|Heartbeat _ | Log_entry _ ->
if Set.mem user_sessions session_id then
(message::messages,user_sessions)
else acc
)
in List.rev user_messages

val messages_for_user :
  string ->
  (Common.t * details) Core.List.t -> (Common.t * details) Core.List.t =
  <fun>


In [60]:
type 'a expr = 
|Base of 'a
|Const of bool
|And of 'a expr list
|Or of 'a expr list
|Not of 'a expr

type 'a expr =
    Base of 'a
  | Const of bool
  | And of 'a expr list
  | Or of 'a expr list
  | Not of 'a expr


In [61]:
type mail_field = To | From | CC |Date |Subject

type mail_field = To | From | CC | Date | Subject


In [62]:
type mail_predicate = {field: mail_field; contains:string}

type mail_predicate = { field : mail_field; contains : string; }


In [63]:
let test field contains = Base {field; contains};;

val test : mail_field -> string -> mail_predicate expr = <fun>


In [64]:
And [ Or [test To "doligez"; test CC "doligez"]]

- : mail_predicate expr =
And
 [Or
   [Base {field = To; contains = "doligez"};
    Base {field = CC; contains = "doligez"}]]


In [66]:
let rec eval expr base_eval = 
let eval' expr = eval expr base_eval in
match expr with
| Base base -> base_eval base
| Const bool -> bool
| And exprs -> List.for_all exprs ~f:eval'
| Or exprs -> List.exists exprs ~f:eval'
| Not expr -> not (eval' expr)

val eval : 'a expr -> ('a -> bool) -> bool = <fun>
